# Chapter13 빅오를 활용한 알고리즘 성능 분석과 개선

### timeit 모듈

- timeit 모듈은 작은 코드 조각의 실행시간 속도를 수백만번씩 실행해서 측정해, 평균 실행시간을 계산할 수 있다.
- 또한 timeit 모듈은 가비지컬렉터를 일시적으로 비활성화하여 더욱 일관된 실행시간을 보장한다.

### timeit 함수의 파라미터
timeit.timeit(stmt='pass', setup='pass', timer=<default timer>, number=1000000, globals=None)
- stmt: 실행측정할 코드 및 함수
- setup: stmt를 실행하기 위해 사전에 필요한 코드나 함수를 선언. setup의 실행 시간은 전체 측정 실행 시간에서 제외됨
- timer: Timer 인스턴스
- number: 선언한 stmt의 수행 횟수. 선언하지 않으면 기본값으로 1000000번이 실행됨

In [13]:
# 1부터 100까지 무작위로 숫자 1천만 개를 생성하는 데 걸리는 시간
import timeit
timeit.timeit('random.randint(1, 100)', 'import random', number = 10000000)

5.720898532999854

In [6]:
# XOR 알고리즘을 이용하여 a와 b를 바꾸는 경우 - 대략 10분의 1초가 걸렸다.
import timeit
timeit.timeit('a, b = 42, 101; a = a^ b; b = a ^ b; a = a ^ b')


0.15642371400099364

In [9]:
# 임시변수 temp를 이용하여 a와 b를 바꾸는 경우 - 가독성을 높이며 시간을 단축시켰다.
import timeit
timeit.timeit('a, b = 42, 101; temp = a; a = b; b = temp')

0.04142180999770062

In [10]:
# 반복가능 언패킹을 사용하여 a와 b를 바꾸는 경우 - 가독성을 높이며 시간을 단축시켰다.
import timeit
timeit.timeit('a, b = 42, 101; a, b = b, a')

0.038437457998952596

> 코드 작성에 참고하면 좋은 규칙 : 일단 작동부터 되게 하고 속도를 개선하라

### cProfile 프로파일러

- 작은 코드 조각은 timeit 모듈이 유용하지만, 전체 함수나 프로그램을 분석하는 데는 cProfile 모듈이 더 효과적이다.
- 프로파일링은 프로그램의 속도, 메모리 사용 등의 여러 측면을 체계적으로 분석한다.
- cProfile 모듈은 프로그램의 개별 함수 호출에 대한 실행시간 프로파일을 생성할 수 있을 뿐만 아니라 프로그램의 실행시간 측정도 가능하다. 코드의 세부적인 측정값을 제공한다.

In [15]:
# 1 부터 백만까지 더하는 함수
import time, cProfile
def addUpNumbers():
    total = 0
    for i in range(1, 1000001):
        total += i

cProfile.run('addUpNumbers()')

         4 function calls in 0.078 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.078    0.078    0.078    0.078 1775512811.py:3(addUpNumbers)
        1    0.000    0.000    0.078    0.078 <string>:1(<module>)
        1    0.000    0.000    0.078    0.078 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




> 각 행은 다양한 함수와 해당 함수에서 소비된 시간을 나타낸다. </br>
> 각 열에 대한 설명
- ncalls : 이 함수가 호출된 횟수
- tottime : 이 함수에서 호출한 다른 함수들의 시간을 제외하고 이 함수 자체에서 소비된 총 시간
- percall : 총 호출 시간을 누적 호출 횟수로 나눈 값
- cumtime : 이 함수와 이 함수가 호출한 모든 다른 함수들에 소비된 누적 시간
- percall : 누적 시간을 호출 횟수로 나눈 값
- filename:lineno(function) : 함수가 들어 있는 파일과 행 번호

## 빅오 알고리즘 분석

### 빅오 차수

  - 차수가 낮은 경우에는 데이터 양이 많아지더라도 코드 실행속도가 천천히 느려지고, 차수가 높은 경우에는 데이터 양이 많아지면 속도가 급격하게 느려진다.
1. O(1) : 상수 시간(가장 낮은 차수) - 빠르다
2. O(log n) : 로그 시간 - 빠르다
3. O(n) : 선형 시간 - 나쁘지 않다
4. O(n log n) : 선형 로그 시간 - 나쁘지 않다
5. O(n^2) : 다항 시간 - 느리다
6. O(2^n) : 지수 시간 - 느리다
7. O(n!) : 팩토리얼 시간(가장 높은 차수) - 느리다

### 책장 정리 사례를 이용한 빅오 차수의 이해

- n은 책장에 꽂힌 책 권수
- 빅오 차수는 책 권수가 많아짐에 따라 작업이 얼마나 더 오래 걸리는지 설명

##### 상수 시간 O(1)
- 책장이 비어있는가?를 알아내는 작업이 상수 시간 연산
- 책장에 책이 몇권 꽂혀 있는지는 아무 상관이 없음(n이 등장하지 않음)
- 책장에 책이 한권이라도 발견되는 순간 멈출 수 있음

##### 로그 시간 O(log n)
- 프로그래밍에서는 대체로 로그의 밑을 2로 가정하는 경우가 많음 (그냥 O(log n)이라고 사용)
- 가나다 순으로 정렬된 책장에서 책을 찾는 작업이 로그 시간 연산
- 책 한 권을 찾으려면 책장 정중앙에 꽂힌 책을 확인하면 됨. 만약 그 책이 내가 찾는 책이면, 작업을 멈춤. 아니라면, 가나다순으로 앞쪽인지 뒤쪽인지 판단 가능. (binary search algorithm)
- 위의 과정에서 검색해야 하는 책의 범위를 절반으로 줄일 수 있음
- 전체 작업량 n의 크기가 두 배로 늘어도 실행시간은 한 단계만 증가한다.

##### 선형시간 O(n)
- 서가에 꽂힌 책을 모두 읽는 작업이 선형 시간 연산
- 모든 책의 페이지가 같다고 가정하면 n의 크기를 두 배로 늘렸을 때 작업시간도 두 배로 늘어나게 됨
- 즉, 실행시간은 n에 정비례해서 증가함

##### 선형 로그 시간 O(n log n)

- 일련의 책을 가나다 순으로 정렬하는 작업이 선형 로그 시간임
- O(n) 과 O(log n)을 곱한 결과 - O(log n)이 걸리는 작업을 n번 수행하는 작업
- 책더미를 빈 책장에 가나다순으로 정리한다고 생각할 때, 책이 꽂힐 위치를 확인하는 작업은 O(log n)이다. 이때 책이 n권 있으므로 O(n log n)
- meregsort, quicksort, heapsort, Timsort 가 대표적인 예시

##### 다항 시간 O(n^2)

- 정렬되지 않은 책장에서 중복된 책이 있는지를 확인하는 작업이 다항 시간 연산
- 만약 책이 100권인 경우
  1. 첫 번째 책을 나머지 99권과 비교
  2. 두 번째 책을 나머지 99권과 비교
  3. 이 작업을 각 책마다 한번씩 100번씩 수행해야 함
  4. 즉, n x n = n^2